Now we can introduce parametrizing to the notebook

datasets = dbutils.jobs.taskValues.get(
    taskKey="Parameter_file_names",   # task key of parameters notebook
    key="output.datasets",
    debugValue=[]
)

queries = []   # keep references to streaming queries


print(datasets)

In [0]:
p_file_name = dbutils.widgets.get("file_name")

df = spark.readStream.format("cloudFiles") \
        .option("cloudFiles.format", "parquet") \
        .option("cloudFiles.schemaLocation",
                f"abfss://bronze@salesstaradlsstorageacc.dfs.core.windows.net/{p_file_name}/checkpoint_{p_file_name}") \
        .load(f"abfss://source@salesstaradlsstorageacc.dfs.core.windows.net/{p_file_name}") 

df.writeStream.format("parquet") \
        .outputMode("append") \
        .option("checkpointLocation",
                f"abfss://bronze@salesstaradlsstorageacc.dfs.core.windows.net/{p_file_name}/checkpoint_{p_file_name}") \
        .option("path",
                f"abfss://bronze@salesstaradlsstorageacc.dfs.core.windows.net/{p_file_name}") \
        .trigger(once=True) \
        .start()
